# Segmenting and Clustering Neighborhoods in Toronto
## By Ryan McEvoy

In this notebook, I will segment and cluster neighborhoods in Toronto. Data will be scraped from Wikipedia and clustered.

### Import libraries
Start by importing the necessary libraries.

In [1]:
import numpy as np
import pandas as pd

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
!pip install sklearn
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

!pip install lxml
import lxml

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'c:\program files\python37\python.exe -m pip install --upgrade pip' command.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'c:\program files\python37\python.exe -m pip install --upgrade pip' command.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'c:\program files\python37\python.exe -m pip install --upgrade pip' command.


I did not import geopy to convert addresses into latitude and longitude values because I will opt to use the csv file provided instead.

## Neighborhood data 
Take the neighborhood data from wikipedia and put into a dataframe

In [2]:
# load data from Wikipedia into dataframe

# use this to identify table from html page
attrs = {"class": "wikitable sortable"}

toronto_df = pd.read_html(io="http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", attrs=attrs)[0]

In [3]:
# Clean data

# Only keep rows with Borough not "Not assigned" and replace missing neighborhoods with borough name
toronto_df = toronto_df[toronto_df["Borough"] != "Not assigned"]
toronto_df["Neighbourhood"] = np.where(toronto_df["Neighbourhood"] != "Not assigned", toronto_df["Neighbourhood"], toronto_df["Borough"])

toronto_df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


## Add Geospatial Latitude and Longitude Data
Using the geopy library to get the latitude and longitude data with Nominatim was not working properly, so the csv file with coordinate information is joined with the existing dataframe on the postal code.

In [4]:
geodata_df = pd.read_csv("http://cocl.us/Geospatial_data")
geodata_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [5]:
toronto_geo_df = pd.merge(left=toronto_df, right=geodata_df, how='inner', on='Postal Code')
toronto_geo_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Explore and cluster the neighborhoods in Toronto
